In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
1# Some of the functions are stored in external files. 
# So that the loader could find those files, we need to explicitly show the path where they are stored
# The path to files may differ in your case... 
import sys
sys.path.append('/content/gdrive/My Drive/AUA/AUA 2020 Spring/Deep Learning/DL_HW4')

In [0]:
import torch
import h5py
import matplotlib.pyplot as plt
from testCases_v4a_Pytorch import *
from dnn_utils_v2_Pytorch import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

torch.set_printoptions(precision=10) # shows 10 digits after the floating dot

torch.manual_seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
def initialize_parameters(n_x, n_h, n_y):
    
    torch.manual_seed(1)
)
    W1 = torch.randn(n_h, n_x)*0.01
    b1 = torch.zeros((n_h, 1))
    W2 = torch.randn(n_y, n_h)*0.01
    b2 = torch.zeros((n_y, 1))

    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters    

In [0]:
parameters = initialize_parameters(3,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = tensor([[ 0.0066135214,  0.0026692410,  0.0006167726],
        [ 0.0062131733, -0.0045190598, -0.0016613023]])
b1 = tensor([[0.],
        [0.]])
W2 = tensor([[-0.0152276848,  0.0038168391]])
b2 = tensor([[0.]])


In [0]:
def initialize_parameters_deep(layer_dims):
    torch.manual_seed(3)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = torch.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = torch.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

In [0]:
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = tensor([[-0.0007664429,  0.0035988151, -0.0078201676,  0.0007152773,
          0.0104602277],
        [ 0.0052218698, -0.0016680915,  0.0005303340,  0.0040460713,
          0.0061129960],
        [ 0.0076039038, -0.0003364326,  0.0099792238,  0.0045918045,
          0.0243639704],
        [-0.0014680817,  0.0048511401,  0.0020523437,  0.0033842819,
          0.0135275396]])
b1 = tensor([[0.],
        [0.],
        [0.],
        [0.]])
W2 = tensor([[ 0.0049518901, -0.0016431698, -0.0067796218, -0.0105910739],
        [ 0.0074769664,  0.0023891740, -0.0039215023,  0.0015191489],
        [-0.0118373316,  0.0053436072, -0.0145102274, -0.0062937401]])
b2 = tensor([[0.],
        [0.],
        [0.]])


In [0]:
def linear_forward(A, W, b):

    Z = W@A + b
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [0]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = tensor([[-1.0822153091, -0.5616527200]])


In [0]:
def linear_activation_forward(A_prev, W, b, activation):

    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [0]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = tensor([[0.1978137195, 0.1715756804]])
With ReLU: A = tensor([[0., 0.]])


In [0]:
def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "relu")
        caches.append(cache)
  
    
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)],parameters['b' + str(L)], activation = "sigmoid")
    caches.append(cache)
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [0]:
X, parameters = L_model_forward_test_case_2hidden()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = tensor([[0.7837043405, 0.7705814242, 0.5695444345, 0.7837043405]])
Length of caches list = 3


In [0]:
def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = (-1/m) * ((Y @ torch.log(AL).t() + (1 - Y) @ torch.log(1 - AL).t()))

    
    cost = torch.squeeze(cost)      # To make sure cost's shape is what we expect.
    assert(cost.shape == ())
    
    return cost

In [0]:
Y, AL = compute_cost_test_case()
print("cost = " + str(compute_cost(AL, Y)))

cost = tensor(0.2797765732)


In [0]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dA_prev = W.t()@dZ
    dW = 1/m *(dZ @ A_prev.t()) 
    db =  1/m * dZ.sum(1, keepdim=True)                  
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [0]:
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = tensor([[-0.8316336274, -0.3802693784, -0.1003148183, -0.6336339712],
        [-0.7257670760, -0.3817131817, -1.8475147486,  0.6139277816],
        [ 0.8353895545,  0.4270242155,  0.9488694072,  0.0108328313],
        [-2.0349035263, -0.9647823572,  0.2986161709, -1.7612611055],
        [ 0.9973999858,  0.5078689456,  0.2876567841,  0.5071032047]])
dW = tensor([[ 0.1632022560,  0.1284346581, -0.1838184297, -0.1737741232,
         -0.0176431704],
        [-0.7802008390, -0.1271579564,  0.0151971653, -0.0429767892,
         -0.0452116542],
        [-0.6705543399, -0.0343476534,  0.1530921608,  0.0376261622,
         -0.1165488437]])
db = tensor([[ 0.4028177261],
        [-0.4397802055],
        [-0.6353005171]])


In [0]:
def linear_activation_backward(dA, cache, activation):
      linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ,linear_cache)
                
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ,linear_cache)
            
    return dA_prev, dW, db

In [0]:
dAL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = tensor([[-0.0254830271,  0.0152399382],
        [-0.0677984580,  0.0405463725],
        [ 0.0857409015, -0.0512767211]])
dW = tensor([[0.0130424909, 0.0462693349, 0.0081148185]])
db = tensor([[0.0130683035]])

relu:
dA_prev = tensor([[-0., 0.],
        [-0., 0.],
        [0., -0.]])
dW = tensor([[0., 0., 0.]])
db = tensor([[0.]])


In [0]:
def L_model_backward(AL, Y, caches):
   
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    dAL = - (torch.div(Y, AL) - torch.div(1 - Y, 1 - AL))
    

    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")
    
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l+1)], current_cache, activation = "relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

In [0]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print_grads(grads)

dW1 = tensor([[-0.0137194376, -0.0071208226, -0.0520380065,  0.2212887704],
        [ 0.0295770243,  0.0153514119,  0.1121860445, -0.4770649970],
        [-0.0151281655, -0.0337276235,  0.0249240343,  0.0240937192]])
db1 = tensor([[-0.1541889310],
        [ 0.3324079216],
        [ 0.0246500261]])
dA1 = tensor([[-0.3083778620,  0.2361355573],
        [ 0.6648158431, -0.5090724230],
        [-0.0643826947,  0.0493000522]])


In [0]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2 # number of layers in the neural network

    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW"+ str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db"+ str(l+1)]

    return parameters

In [0]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = tensor([[ 0.3119692206, -0.2410473526, -0.3283980787, -1.1436653137],
        [ 1.0398452282, -0.4964017868,  0.5393572450,  0.4398358464],
        [-0.2483879030, -1.1404219866,  1.2664123774,  0.6238473058]])
b1 = tensor([[-1.2400170565],
        [-1.2711932659],
        [-0.1447267979]])
W2 = tensor([[-0.9417777061,  0.5254638195, -0.3200427592]])
b2 = tensor([[-0.0250098575]])
